In [4]:
%load_ext cython

import multiprocessing 
import numpy as np
import threading
import queue

### Tutorial on using Cython to lift Python GIL

In [5]:
input_range = range(int(1e6), int(2e6), int(5e4))
ncpus = multiprocessing.cpu_count()
print("We have {} cores to work on!".format(ncpus))

We have 8 cores to work on!


### Single Thread

In [6]:
%%cython

from libc.math cimport ceil, sqrt


cdef inline int _is_prime(int n) nogil:
   """return a boolean, is the input integer a prime?"""
   if n == 2:
       return True
   cdef int max_i = <int>ceil(sqrt(n))
   cdef int i = 2
   while i <= max_i:
      if n % i == 0:
          return False
      i += 1
   return True


cdef unsigned long _sum_primes(int n) nogil:
   """return sum of all primes less than n """
   cdef unsigned long i = 0
   cdef int x
   for x in range(2, n):
       if _is_prime(x):
           i += x
   return i


def sum_primes(int n):
    with nogil:
        result = _sum_primes(n)
    return result

In [7]:
%%time

for i in input_range:
    print(sum_primes(i), end=' ', flush=True)
print()

37550402023 41276629127 45125753695 49161463647 53433406131 57759511224 62287995772 66955471633 71881256647 76875349479 82074443256 87423357964 92878592188 98576757977 104450958704 110431974857 116581137847 122913801665 129451433482 136136977177 
CPU times: user 18.1 s, sys: 0 ns, total: 18.1 s
Wall time: 18 s


### Multiprocessing Threading

In [8]:
%%time

### We need to define a worker function that fetches jobs from the queue.
def worker(q):
    while True:
        try:
            x = q.get(block=False)
            print(sum_primes(x), end=' ', flush=True)
        except queue.Empty:
            break

### Create the queue, and fill it with input values
work_queue = queue.Queue()
for i in input_range:
    work_queue.put(i)

### Start a number of threads
threads = [
    threading.Thread(target=worker, args=(work_queue,))
    for i in range(ncpus)]

for t in threads:
    t.start()

### Wait until all of them are done
for t in threads:
    t.join()

print()

37550402023 49161463647 41276629127 62287995772 66955471633 53433406131 45125753695 57759511224 71881256647 76875349479 82074443256 92878592188 87423357964 98576757977 110431974857 104450958704 116581137847 122913801665 129451433482 136136977177 
CPU times: user 21.4 s, sys: 519 ms, total: 21.9 s
Wall time: 3.42 s
